In [1]:
from pathlib import Path
import cv2
import sys
import numpy as np
from skimage import feature, measure
import live_cell_tracker
from live_cell_tracker.livecell_segment import datasets
import live_cell_tracker.livecell_segment
from live_cell_tracker import livecell_segment
import live_cell_tracker.core.utils
from tqdm import tqdm
seg_path = r"""D:\xing-vimentin-dic-pipeline\src\cxa_segmentation\notebooks\segmentation_results\detectron_model3\restart_day0_Group 1_wellA1_RI_MIP_stitched"""
label_seg_imgs = livecell_segment.datasets.LiveCellImageDataset(seg_path, ext="png")
dir_path = Path(
    r"D:\xing-vimentin-dic-pipeline\src\cxa_segmentation\cxa-data\june_2022_data_8bit_png\restart_day0_Group 1_wellA1_RI_MIP_stitched"
)
raw_imgs = livecell_segment.datasets.LiveCellImageDataset(dir_path, ext="png")

288 png img file paths loaded: 
288 png img file paths loaded: 


In [2]:
def gen_detections_for_sort_tracker(label_mask):
    """
        generate detections for SORT tracker. detections: [x1, y1, x2, y2, score]
    Parameters
    ----------
    label_mask :
        an image
    Returns
    -------
    A list of (x1, y1, x2, y2, score]) for each object detected
    """    
    regionprops_skimge = measure.regionprops(label_mask)
    boxes = live_cell_tracker.core.utils.get_bbox_from_regionprops(regionprops_skimge)
    detections = np.array([list(bbox) + [1] for bbox in boxes])
    return detections

import live_cell_tracker.track.sort_tracker
tracker = live_cell_tracker.track.sort_tracker.Sort() 

all_track_bbs = []
for i, img in enumerate(tqdm(label_seg_imgs)):
    detections = gen_detections_for_sort_tracker(img)
    track_bbs_ids = tracker.update(detections)
    all_track_bbs.append(track_bbs_ids)
    # for testing
    # if i > 5:
    #     break

100%|██████████| 288/288 [00:10<00:00, 28.50it/s]


In [11]:
all_track_bbs[0].shape

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)


(27, 5)

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)


convert to int

In [54]:
all_track_bbs = [track_bbs.astype(np.int32).tolist() for track_bbs in all_track_bbs]
all_track_bbs[0]

[[1, 1396, 67, 1581, 27],
 [734, 2114, 914, 2264, 26],
 [382, 274, 469, 390, 25],
 [1385, 1310, 1445, 1363, 24],
 [1395, 1278, 1450, 1323, 23],
 [1829, 1590, 1906, 1619, 22],
 [144, 1333, 231, 1391, 21],
 [285, 270, 342, 358, 20],
 [60, 1266, 148, 1329, 19],
 [90, 1414, 166, 1473, 18],
 [1850, 1369, 1904, 1419, 17],
 [12, 1234, 301, 1400, 16],
 [271, 465, 351, 529, 15],
 [7, 1234, 115, 1304, 14],
 [760, 2236, 987, 2377, 13],
 [1012, 2321, 1102, 2385, 12],
 [103, 1304, 322, 1406, 11],
 [1733, 1571, 1989, 1690, 10],
 [988, 2282, 1140, 2402, 9],
 [780, 2238, 821, 2253, 8],
 [1783, 1494, 2034, 1574, 7],
 [69, 424, 607, 694, 6],
 [350, 180, 584, 413, 5],
 [1789, 1256, 2015, 1480, 4],
 [48, 1391, 280, 1543, 3],
 [1297, 1159, 1571, 1481, 2],
 [152, 190, 359, 445, 1]]

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)


### Save track bbox results to json for later development

In [55]:
dest_track_bbs_path = "detectron_model3_all_track_bbs-restart_day0_Group 1_wellA1_RI_MIP_stitched.json"
import json
with open(dest_track_bbs_path, "w+") as out_f:
    json.dump(all_track_bbs, out_f)


int

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)


## Visualize Track results

In [ ]:
with open(dest_track_bbs_path, "r") as in_f:
    json_in = json.load(in_f)
type(json_in[0][0][0])

In [56]:
all_track_bbs = [np.array(track_bbs, dtype=int) for track_bbs in json_in]

In [57]:
all_track_bbs[0].shape

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)


(27, 5)

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)


In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
idx = 2
bboxes = all_track_bbs[idx]
fig, ax = plt.subplots()
ax.imshow(raw_imgs[idx])
for bbox in bboxes:
    # Create a Rectangle patch
    rect = patches.Rectangle((bbox[1], bbox[0]),  (bbox[3] - bbox[1]), (bbox[2] - bbox[0]), linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
# plt.show()


In [8]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from live_cell_tracker.core import SingleCellTrajectory, SingleCellStatic, single_cell

def visualize_bbox_trajectory(track_id, all_track_bbs, raw_imgs, save_path="./tmp.gif", track_id_col = 4):
    fig, ax = plt.subplots()
    def init():
        return []
    def update(frame):
        frame_idx, raw_img, bbox, img_crop = frame
        ax.cla()
        frame_text = ax.text(-10, -10, "frame: {}".format(frame_idx), fontsize=10, color="red", ha="center", va="center")
        ax.imshow(img_crop)
        return []

    frame_data = []
    for frame_idx, track_bbs in enumerate(all_track_bbs):
        img = raw_imgs[frame_idx]
        for bbox in track_bbs:
            if bbox[track_id_col] != track_id:
                continue
            min_x, max_x, min_y, max_y = int(bbox[0]), int(bbox[2]), int(bbox[1]), int(bbox[3])
            img_crop = img[min_x:max_x, min_y:max_y]
            frame_data.append((frame_idx, img, bbox, img_crop))

    ani = FuncAnimation(fig, update, frames=frame_data,
                init_func=None, blit=True)
    print("saving to: %s..." % save_path)
    ani.save(save_path)

output_path = Path("./track_tmp")
output_path.mkdir(exist_ok=True)

id_to_sc_trajs = {}
for timeframe, objects in enumerate(all_track_bbs):
    for obj in objects:
        track_id = obj[-1]
        if  not (track_id in id_to_sc_trajs):
            new_traj = SingleCellTrajectory(raw_imgs)
            id_to_sc_trajs[track_id] = new_traj

        sc = SingleCellStatic(timeframe)
        _traj = id_to_sc_trajs[track_id]
        _traj.add_timeframe_data(timeframe, sc)

# visualize_bbox_trajectory(track_id, all_track_bbs, raw_imgs, save_path=output_path / "track_{}.gif".format(track_id))

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 35,
 36,
 51,
 53,
 54,
 56,
 58,
 59,
 65,
 66,
 72,
 76,
 79,
 87,
 88,
 93,
 101,
 103,
 104,
 109,
 112,
 116,
 118,
 132,
 134,
 135,
 136,
 139,
 143,
 148,
 153,
 154,
 156,
 158,
 159,
 165,
 173,
 185,
 192,
 194,
 200,
 224,
 230,
 231,
 236,
 246,
 258,
 259,
 261,
 264,
 267,
 278,
 279,
 284,
 288,
 289,
 291,
 292,
 295,
 298,
 299,
 304,
 306,
 310,
 311,
 313,
 317,
 320,
 326,
 330,
 345,
 357,
 360,
 366,
 376,
 378,
 384,
 385,
 394,
 402,
 405,
 411,
 414,
 415,
 416,
 421,
 428,
 456,
 461,
 462,
 465,
 468,
 479,
 482,
 487,
 489,
 495,
 496,
 500,
 503,
 504,
 506,
 507,
 510,
 512,
 518,
 522,
 523,
 527,
 531,
 532,
 533,
 535,
 538,
 539,
 543,
 544,
 545,
 548,
 551,
 557,
 563,
 565,
 570,
 576,
 579,
 580,
 589,
 595,
 599,
 600,
 608,
 618,
 619,
 620,
 629,
 640,
 645,
 647,
 654,
 660,
 661,
 668,
 675,
 

C:\Users\dummy\AppData\Local\Temp\ipykernel_1184\3789225958.py:13: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
  ax.imshow(img_crop)
